<a href="https://colab.research.google.com/github/Sibu08/movierecommendation/blob/master/MovieLens25M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import gc
import numpy as np
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

In [0]:
ratings = pd.read_csv('/content/drive/My Drive/DS Project 20M/ratings.csv')

In [5]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [0]:
movies = pd.read_csv('/content/drive/My Drive/DS Project 20M/movies.csv')

In [0]:
total=ratings.merge(movies,on='movieId',how='inner')
total['genres']=total['genres'].apply(lambda x: x.split('|'))

In [8]:
gc.collect()

0

In [9]:
total

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]"
1,3,296,5.0,1439474476,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]"
2,4,296,4.0,1573938898,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]"
3,5,296,4.0,830786155,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]"
4,7,296,4.0,835444730,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]"
...,...,...,...,...,...,...
25000090,162358,200192,2.0,1553453039,Den frusna leoparden (1986),[(no genres listed)]
25000091,162358,200194,2.0,1553453843,Tough Luck (2004),"[Action, Adventure, Thriller]"
25000092,162386,139970,3.5,1549215965,I Don't Speak English (1995),[Comedy]
25000093,162386,200726,4.0,1554651417,The Graduates (1995),"[Children, Drama]"


In [0]:
test=pd.DataFrame(total.groupby(['userId']).agg(movie_count=pd.NamedAgg(column='movieId', aggfunc='count')))

In [0]:
test = test.sort_values('movie_count',ascending=False)

In [19]:
test.describe()

,movie_count
count,162541.000000
mean,153.807932
std,268.047590
min,20.000000
25%,36.000000
50%,71.000000
75%,162.000000
max,32202.000000


In [0]:
test = test[test['movie_count'] > 200]

In [0]:
test = test.reset_index()

In [22]:
test

,userId,movie_count
0,72315,32202
1,80974,9178
2,137293,8913
3,33844,7919
4,20055,7488
...,...,...
32843,156456,201
32844,115756,201
32845,6761,201
32846,68693,201


In [0]:
total=total.merge(test,on='userId',how='inner')

In [0]:
total.drop(columns = ['movie_count'], inplace = True)

In [26]:
total

,userId,movieId,rating,timestamp,title,genres
0,3,296,5.0,1439474476,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]"
1,3,1217,5.0,1439474455,Ran (1985),"[Drama, War]"
2,3,1653,5.0,1439474631,Gattaca (1997),"[Drama, Sci-Fi, Thriller]"
3,3,4308,3.0,1439472792,Moulin Rouge (2001),"[Drama, Musical, Romance]"
4,3,5952,4.0,1439472616,"Lord of the Rings: The Two Towers, The (2002)","[Adventure, Fantasy]"
...,...,...,...,...,...,...
16063553,115410,140349,2.5,1453051430,Gabriel Iglesias: I'm Not Fat... I'm Fluffy (2...,[Comedy]
16063554,115410,133205,5.0,1453051977,Doug Stanhope: Beer Hall Putsch (2013),[Comedy]
16063555,115410,89584,3.0,1453051529,Skeletons (2010),"[Comedy, Fantasy]"
16063556,115410,165155,5.0,1484935807,Russell Peters: Almost Famous (2016),[Comedy]


In [50]:
total.describe()['timestamp']['75%']

1455488882.0

In [0]:
temp = total[['userId','movieId','timestamp']]

In [0]:
temp1 = temp.copy()

In [46]:
temp1

,userId,movieId,timestamp
0,3,296,1439474476
1,3,1217,1439474455
2,3,1653,1439474631
3,3,4308,1439472792
4,3,5952,1439472616
...,...,...,...
16063553,115410,140349,1453051430
16063554,115410,133205,1453051977
16063555,115410,89584,1453051529
16063556,115410,165155,1484935807


In [0]:
temp1['class'] = temp1['timestamp'].apply(lambda x: 1 if x < 1455488882.0 else 2)

In [52]:
temp1

,userId,movieId,timestamp,class
0,3,296,1439474476,1
1,3,1217,1439474455,1
2,3,1653,1439474631,1
3,3,4308,1439472792,1
4,3,5952,1439472616,1
...,...,...,...,...
16063553,115410,140349,1453051430,1
16063554,115410,133205,1453051977,1
16063555,115410,89584,1453051529,1
16063556,115410,165155,1484935807,2


In [0]:
del ratings

In [54]:
gc.collect()

229

In [58]:
temp1.groupby(['userId','class'])['movieId'].count()

userId  class
3       1        428
        2        228
4       2        242
12      1        736
13      1        412
                ... 
162521  2         16
162524  1        229
162529  1        382
162533  1        367
162534  2        487
Name: movieId, Length: 37023, dtype: int64

## ***USER GENRE MATRIX***

In [0]:
genres = []
for i in movies.index:
    genres.extend(movies['genres'][i].split('|'))

In [0]:
genres = list(set(genres))
genres.remove('(no genres listed)')
len(genres)

19

In [0]:
movies_temp = movies.copy()

In [0]:
movies_temp = movies_temp.set_index('movieId')

In [0]:
movie_genre = pd.DataFrame(columns = genres, index = movie_ids)

In [0]:
movie_genre[genres] = 0.0

In [0]:
for i in movie_ids:
    for j in movies_temp['genres'][i].split('|'):
        try:
            movie_genre[j][i] = 1
        except KeyError:
            pass

In [0]:
movie_genre

,Film-Noir,Horror,Romance,Crime,War,Comedy,Adventure,Documentary,Western,Action,Mystery,Fantasy,Animation,Thriller,Children,Drama,IMAX,Musical,Sci-Fi
356,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
318,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
296,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
593,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2571,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
5418,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0
924,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
778,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


## ***Timestamp***

In [0]:
test.describe()

,count_rate,count_num
count,100.000000,100.000000
mean,3.853482,41710.060000
std,0.329634,12612.478649
min,2.871660,28516.000000
25%,3.687729,32083.250000
50%,3.948924,37340.000000
75%,4.091465,48508.250000
max,4.413576,81491.000000


In [0]:
test_75 = total[total['timestamp'] <= total.describe()['timestamp']['75%']]

In [0]:
test_25 = total[total['timestamp'] > total.describe()['timestamp']['75%']]

In [0]:
len(test_75['userId'].unique())

121537

In [0]:
len(test_25['userId'].unique())

41302

In [0]:
common = list(set(test_75['userId'].unique()).intersection(set(test_25['userId'].unique())))

In [0]:
len(common)

3469

In [0]:
test_75 = test_75[test_75.userId.isin(common)]

In [0]:
test_25 = test_25[test_25.userId.isin(common)]

In [0]:
test_75

,userId,movieId,rating,timestamp,title,genres,count_rate,count_num
37,76,296,4.5,1285219680,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",4.188912,79672
140,284,296,4.5,1431924674,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",4.188912,79672
173,357,296,3.5,1367156671,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",4.188912,79672
179,368,296,5.0,1402798200,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",4.188912,79672
207,426,296,4.5,1281844645,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",4.188912,79672
...,...,...,...,...,...,...,...,...
4170909,162147,1193,4.5,1378776462,One Flew Over the Cuckoo's Nest (1975),[Drama],4.218662,36058
4170951,162297,1193,4.5,1306952024,One Flew Over the Cuckoo's Nest (1975),[Drama],4.218662,36058
4170967,162390,1193,4.0,1428093132,One Flew Over the Cuckoo's Nest (1975),[Drama],4.218662,36058
4170994,162501,1193,5.0,1415410819,One Flew Over the Cuckoo's Nest (1975),[Drama],4.218662,36058


In [0]:
test75_groupby = pd.DataFrame(test_75.groupby(['userId', 'timestamp'])['movieId'].unique())

In [0]:
user_movie_pred2 = {}
for i in common:
    user_movie_pred2[i] = []

In [0]:
num = 5
for user in common:
    #print(user)
    count = 0
    try:
      while len(user_movie_pred2[user]) != 15:
          print(user, count)
          movie_id = (test75_groupby.loc[user])[::-1].iloc[count]['movieId']
          for i in movie_id:
              array = (cosine_similarity(np.array(movie_genre.loc[i]).reshape(1, 19), movie_genre))[0].argsort()[-num:][::-1]
              user_movie_pred2[user].extend(test[test.index.isin(array.tolist())].movieId.tolist())
              if num > 1:
                  num -= 1
          user_movie_pred2[user] = list(set(user_movie_pred2[user]).difference(set(test_75[test_75['userId'] == user].movieId.tolist())))
          count += 1
    except IndexError:
      pass

Streaming output truncated to the last 5000 lines.
114423 69
114423 70
114423 71
81657 0
81657 1
81657 2
81657 3
81657 4
81657 5
81657 6
81657 7
81657 8
81657 9
81657 10
81657 11
81657 12
81657 13
81657 14
81657 15
81657 16
81657 17
81657 18
81657 19
81657 20
81657 21
81657 22
81657 23
81657 24
81657 25
81657 26
81657 27
81657 28
81657 29
81657 30
81657 31
81657 32
81657 33
81657 34
81657 35
81657 36
81657 37
81657 38
81657 39
81657 40
81657 41
81657 42
81657 43
81657 44
81657 45
81657 46
81657 47
81657 48
81657 49
81657 50
81657 51
81657 52
81657 53
81657 54
81657 55
81657 56
81657 57
81657 58
81657 59
81657 60
81657 61
81657 62
81657 63
81657 64
81657 65
81657 66
81657 67
139005 0
139005 1
139005 2
139005 3
139005 4
139005 5
139005 6
139005 7
139005 8
139005 9
139005 10
139005 11
139005 12
139005 13
139005 14
139005 15
139005 16
139005 17
139005 18
139005 19
139005 20
139005 21
139005 22
139005 23
139005 24
139005 25
139005 26
139005 27
139005 28
139005 29
139005 30
139005 31
139005 

In [0]:
for user in user_movie_pred2:
    match = set(user_movie_pred2[user]).intersection(set(test_25[test_25['userId'] == 15].movieId.tolist()))
    if len(match) > 0:
      print(user)
      print(list(match), len(match))

In [0]:
user_movie_pred2

{16388: [344, 1196, 4973],
 24582: [1240, 4993, 5349],
 106507: [],
 24588: [1240, 597],
 40972: [3147, 597],
 90128: [1, 2028],
 57362: [3147, 4995, 5349, 47],
 139283: [3147, 1291, 4995],
 122899: [1240, 1, 3147, 1036],
 90133: [1240, 3147, 47],
 24598: [3147, 1196, 47],
 139289: [1089, 3147],
 81957: [3147, 4995, 2028, 5349],
 32808: [],
 131113: [1240, 4995, 3147, 1196],
 131116: [1240, 1291],
 155694: [1089, 4995, 1291, 3147, 597, 1240],
 147503: [1580],
 73775: [4995, 296, 3147, 1036, 597, 1240],
 147505: [4995, 5349, 1580, 2028, 1240],
 32818: [1, 4995, 5349, 3147, 1196, 47],
 32823: [4995, 5349, 1291, 3147, 1036, 47, 1240],
 57399: [1089, 4995, 5349, 3147, 1036, 597, 1240],
 139321: [1089, 1, 4995, 5349, 3147],
 16444: [1240, 5349],
 139325: [1, 3147, 1036, 597, 1240],
 8253: [3147, 2028, 1036, 47, 597, 1240],
 147521: [1, 4995, 3147, 1580, 47],
 65602: [1089, 5349, 1036, 597, 1240],
 41027: [1240],
 73797: [3147],
 76: [1291, 3147, 597, 1240],
 32845: [4995, 1036],
 16465: [49